In [96]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [1]:
PATH = '/Users/maksimsadkov/SQL T+ test'

In [2]:
import psycopg2
import pandas as pd

import os

In [3]:
def get_connection(password, dbname='postgres', user='postgres', host='localhost'):
    try: 
        conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
        return conn
    except:
        return False

In [4]:
def execute_query(connection, query):
    try:
        with connection.cursor() as cur:
            cur.execute(query)
            res = cur.fetchall()
            colnames = [desc[0] for desc in cur.description]
            return res, colnames
    except Exception as e:
        return e, False
   

In [5]:
def create_csv_report(df, path):
    try:
        df.to_csv(path)
    except Exception as e:
        return e

In [6]:
def create_df(data, column_names):
    return pd.DataFrame(data, columns=column_names)

In [7]:
conn = get_connection('123')

In [8]:
query_1 = '''
select cpn.per_id, cpn.entity_name, cap.acct_id , cap.acct_rel_type 
from 
	ci_acct_per cap
	left join ci_per_name cpn on cap.per_id = cpn.per_id ;
'''

query_2 = '''
select cpn.entity_name as owner_name, cpn2.entity_name as family_member_name
from 
	ci_per_name cpn 
	left join ci_acct_per cap on cap.per_id = cpn.per_id
	left join ci_acct_per cap2 on cap2.acct_id = cap.acct_id 
	left join ci_per_name cpn2 on cpn2.per_id = cap2.per_id
WHERE cap.main_cust_sw  = 'Y' and cap.acct_rel_type = 'FMLMMBRA';
'''

query_3 = '''
select cpn1.per_id,cpn2.entity_name 
from (select cpn.per_id
	from ci_acct_per cap 
	left join ci_per_name cpn on cpn.per_id = cap.per_id
	group by cpn.per_id
	having count(cap.acct_id) > 1) as cpn1
left join ci_per_name cpn2 on cpn2.per_id = cpn1.per_id;
'''

query_4 = '''
select ca.acct_id, cac.char_val, cac.effdt
from ci_acct ca 
left join(
	select * from (
		select *,
		row_number() over(partition by acct_id order by effdt DESC) as row_num	
		from (
			select * from ci_acct_char
			where char_type_cd = 'PENI-PRM'
			) peni_prm_cac
			) cac_rang
	where row_num = 1) cac on cac.acct_id = ca.acct_id;
'''

query_5 = '''
select DISTINCT cpn.per_id, cpn.entity_name
from ci_acct_char cac 
left join ci_acct_per cap on cap.acct_id  = cac.acct_id 
left join ci_per_name cpn on cpn.per_id  = cap.per_id 
left join ci_acct_char cac2  on cap.acct_id  = cac2.acct_id 
where cac.char_type_cd = 'OTDELEN' and cac.effdt <= '2015-01-01' and cap.acct_rel_type = 'OSNOV' and cac2.effdt > '2015-01-01'
and cac.char_val <> cac2.char_val;
'''

query_6 = '''
select sub_q1.acct_id
from (
	select acct_id from (
		select * from ci_sa cs 
		where sa_type_cd != 'PEREPL'
	) cs2
	group by acct_id) sub_q1
left join (
	select * from ci_sa cs 
	where sa_type_cd = 'PEREPL'
	) sub_q2 on sub_q1.acct_id = sub_q2.acct_id
where sub_q2.acct_id is null or sub_q2.acct_id = '';
'''

query_7 = '''
select *
from ci_acct ca 
order by setup_dt 
limit 1;
'''

In [9]:
query_list = [query_1, query_2, query_3, query_4, query_5, query_6, query_7]

In [13]:
for idx, query in enumerate(query_list):
    res = execute_query(conn, query)
    if res[1] != False:
        data, col_names = res[0], res[1]
        df = create_df(data, col_names)
        csv = create_csv_report(df, f'report_{idx+1}')